**Para la tarea:** Modela el juego de las torres de Hanói como un problema de búsqueda.

Para las torres de Hanoi podemos pensar en cada disco siendo representado por una letra, siendo `A` el disco más grande, y `G` el más chico. A su ves pensando en `/` como el espacio entre cada disco.
Así, el estado inicial: `ABCDEFG//`

Conjunto de todas las acciones consiste en solo dos: [x>,x>>]
La letra "x" representa a cualquier letra que se desee mover (por lo que en una escritura menos general serían 14 movimientos), mientras que ">" y ">>" representan el moverse a la primera torre vecina hacia la derecha y a la segunda respectivamente. Si el disco no tuviera vecino hacia la derecha, por ejemplo, si estuviera en la torre más a la derecha y tuviera la acción "x>" iría a la primer torre, si tuviera la accion "x>>" llegaría a la de enmedio, y así.

El costo de cada acción será unitario.

El sucesor al que se llega con cada acción sería una distribución de letras y diagonales un poco diferente, variando en una letra en cada caso. por ejemplo:

`ABCDEFG//` $\rightarrow$ `G>` $\rightarrow$ `ABCDEF/G/`

Se llega a un estado final cuando Todas las letras estan juntas y en orden alfabetico.
***

**Para la tarea:** Escribe un programa de Python que genere un tablero de Sudoku resuelto de forma aleatoria de tamaño $9\times9$.  No busques algoritmos para lograr esto, en su lugar, plantea una idea clave para resolver el problema y asegúrate que tu implementación sea lo más clara y entendible que puedas.

In [6]:
import random
import numpy as np

In [7]:
def initial_board():
    return np.zeros((9,9), dtype = int)

In [8]:
def is_valid(board, row, col, number):
        for i in range(9):
            if board[row][i] == number or board[i][col] == number:
                return False
        box_row = row - row % 3
        box_col = col - col % 3
        for i in range(box_row, box_row+3):
            for j in range(box_col, box_col+3):
                if board[i][j] == number:
                    return False
        return True

In [9]:

def backtrack(board):
    numbers = [i for i in range(1,10)]
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                random.shuffle(numbers)
                for number in numbers:
                    if is_valid(board, row, col, number):
                        board[row][col] = number
                        if backtrack(board):
                            return True
                        board[row][col] = 0
                return False
    return True

In [10]:
def solved_board():
    board = initial_board()
    backtrack(board)
    print(np.matrix(board))

In [11]:
solved_board()

[[4 3 5 2 7 1 8 9 6]
 [8 9 7 3 6 5 4 2 1]
 [2 6 1 4 8 9 3 5 7]
 [3 1 9 6 4 7 5 8 2]
 [6 4 8 9 5 2 1 7 3]
 [5 7 2 1 3 8 6 4 9]
 [1 8 4 7 2 6 9 3 5]
 [9 2 3 5 1 4 7 6 8]
 [7 5 6 8 9 3 2 1 4]]


***
**Para la tarea:** Escribe una implementación de `backtrackingSearch` que sea funcionalmente equivalente a la implementación de arriba sin utilizar variables fuera del ámbito local de una función.

In [79]:
class TransportationProblem(object):
    def __init__(self, n):
        self.n = n
    
    def initialState(self):
        return 1
    
    def isEnd(self, state):
        return state == self.n
    
    def actions(self, state):
        moves = []
        if state + 1 <= self.n:
            moves.append('walk')
        if state * 2 <= self.n:
            moves.append('tram')
        return moves
    
    def cost(self, state, action):
        if action == 'walk':
            return 1
        if action == 'tram':
            return 2
    
    def successor(self, state, action):
        if action == 'walk':
            return state + 1
        if action == 'tram':
            return state * 2

In [80]:
def edges(problem, state):
    return [
        (action,
         problem.successor(state, action),
         problem.cost(state, action))
        for action in problem.actions(state)
    ]

In [97]:
problem  = TransportationProblem(n=100)

In [94]:
def backtrackingSearch(problem):
    def findBest(state, path, cost, bestCost, bestPath):
        if problem.isEnd(state):
            if cost < bestCost:
                bestCost = cost
                bestPath = path
            return bestCost, bestPath
        for action, next_state, next_cost in edges(problem, state):
            new_path = path + [(action, next_state, next_cost)]
            new_cost = cost + next_cost
            bestCost, bestPath = findBest(next_state, new_path, new_cost, bestCost, bestPath)
        return bestCost, bestPath
    initial_state = problem.initialState()
    bestCost, bestPath = findBest(initial_state, [], 0, float('+inf'), None)
    return bestCost, bestPath

In [95]:
backtrackingSearch(problem)

(6,
 [('walk', 2, 1),
  ('walk', 3, 1),
  ('walk', 4, 1),
  ('walk', 5, 1),
  ('tram', 10, 2)])

**Para la tarea:** Observa que tanto la implementación de `futureCosts` como de `dynamicProgramming` iteran sobre las aristas del estado actual. Programa una mejor implementación combinando las ideas de ambas funciones.

In [96]:
def dynamicProgramming(problem, C):
    state = problem.initialState()
    bestCost = C[state]
    bestPath = []
    while not problem.isEnd(state):
        for action, next_state, next_cost in edges(problem, state):
            if C[state] == next_cost + C[next_state]:
                bestPath.append((action, next_state, next_cost))
                state = next_state
                break
    return bestCost, bestPath


def futureCosts(problem):
    C = {}
    def futureCost(state):
        if state in C:
            return C[state]
        if problem.isEnd(state):
            C[state] = 0
        else:
            C[state] = min(next_cost + futureCost(next_state)
                           for action, next_state, next_cost
                           in edges(problem, state))
        return C[state]
    futureCost(problem.initialState())
    return C

In [101]:
def dynamicProgrammingUpgrade(problem):
    C = {}
    bestPath = []

    def futureCost(state):
        if state in C:
            return C[state]
        if problem.isEnd(state):
            C[state] = 0
        else:
            C[state] = min(next_cost + futureCost(next_state)
                           for _, next_state, next_cost
                           in edges(problem, state))
        return C[state]

    state = problem.initialState()
    bestCost = futureCost(state)
    bestPath = []
    while not problem.isEnd(state):
        for action, next_state, next_cost in edges(problem, state):
            if C[state] == next_cost + C[next_state]:
                bestPath.append((action, next_state, next_cost))
                state = next_state
                break
    return bestCost, bestPath

In [102]:
%%timeit -r 1 -n 1
print(dynamicProgrammingUpgrade(problem))
print()

(13, [('walk', 2, 1), ('walk', 3, 1), ('tram', 6, 2), ('tram', 12, 2), ('tram', 24, 2), ('walk', 25, 1), ('tram', 50, 2), ('tram', 100, 2)])

9.89 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


**Para la tarea:** ¿Por qué el algoritmo de Dijkstra no puede trabajar con pesos negativos? ¿Qué pasa si le sumamos a todos los pesos el peso mínimo de la gráfica? Presenta una gráfica dirigida ponderada en donde esta "solución" no funciona.

Porque el algoritmo en cada paso escoge el nodo con el peso más pequeño, el más cercano, o el de menor costo. Si se tuvieran pesos negativos (en términos de distancia se podría decir que se retrocede) el algoritmo lo sigue considerando como un avance, y contabiliza esto en el total de la distancia, por lo tanto se llegaria a una distancia, coste, etc. mayor a la distancia optima. También se podría llegar a un bucle del que no se podría salir.

Si le sumamos el peso minimo a una grafica obtendriamos un coste cero en donde estaba ese valor, el algoritmo seguiria tomando un camino con este nodo como el camino optivo (cuando en realidad puede que no lo sea).